# Import Libraries

In [1]:
import requests
from lxml import html
import os
import zipfile

# Scrape the login page

In [2]:
def scrape_download(username, password):
    # Fill in login details here to be posted to the login form.
    payload1 = {
        "username": username,
        "password": password
    }
    login_url = "https://freddiemac.embs.com/FLoan/secure/auth.php"

    payload2 = {
        "accept": "Yes",
        "action": "acceptTandC",
        "acceptSubmit": "Continue"
    }
    download_url = "https://freddiemac.embs.com/FLoan/Data/download.php"

    # Use 'with' to ensure the session context is closed after use.
    with requests.Session() as s:
        page = s.post(login_url, data=payload1, headers = dict(referer = login_url))
        # print the html returned or something more intelligent to see if it's a successful login page.
        print (page.text)

        page = s.post(download_url, payload2, headers = dict(referer = download_url))
        # print the html returned or something more intelligent to see if it's check the Terms and Conditions checkbox.
        print (page.text)
        
        result = html.fromstring(page.content)
        links = result.findall(".//a")

        ## Download the sample files

        ### make a directory
        os.makedirs('zip_file_folder', mode=0o777)

        download_url = "https://freddiemac.embs.com/FLoan/Data/"
        for link in links:
            href = link.get("href")
            ## add conditions to download files you need
            if "sample" in href:
                url = download_url + href
                r = s.get(url, stream = True)
                with open(os.path.join("zip_file_folder", link.text), "wb") as f:
                    for chunk in r.iter_content(chunk_size = 1024):
                        if chunk:
                            f.write(chunk)

In [3]:
scrape_download("liang.g@husky.neu.edu", "n2~4J`rH")

<!DOCTYPE html>
<html>

<head>
<title>Freddie Mac Loan-Level Dataset</title>
<link rel='stylesheet' type='text/css' href='/FLoan/app_corp.css'>
<link rel='stylesheet' type='text/css' href='/FLoan/empty.css'>
<link rel='stylesheet' type='text/css' href='/FLoan/empty2.css'>
</head>
<body class=''>
<center>
<div id='content-display' class='' >
<div id='content-main' style='width:550px; text-align:left'>
<P><H2>Loan-Level Dataset</H2></b>
<form action='/FLoan/Data/download.php' method='post' class='fmform'><P><INPUT NAME=accept TYPE=checkbox value=Yes>I have read and agree to the <A HREF='/FLoan/HistoricalDataTerms.html' Target='_blank'>Terms and Conditions</A>.<input type=hidden name=action value=acceptTandC><P><input type=submit class='fmSubmit' name=acceptSubmit value='Continue'></P></form><form action='/FLoan/secure/login.php' method='post' class='fmform'>
<input type=hidden name=option value='logout'>
<p class='field clearfix'>
<input type=submit name=logout value='Log Out' class='fmS

# Unzip the sample files

In [4]:
def unzip(zip_path, unzip_path):
    zip_files = os.listdir(zip_path)
    for file in zip_files:
        zip_ref = zipfile.ZipFile(os.path.join(zip_path, file), "r")
        zip_ref.extractall(os.path.join(unzip_path, file))
        zip_ref.close()

In [5]:
unzip("zip_file_folder", "unzip_file_folder")